In [ ]:
HTML(read(open("style.css"), String))

# Minimax
A program to play a game of chess using the Minimax algorithm. The Minimax algorithm calculates all different possibilities of the current position and chooses the best move. It assumes that all players try to play the best possible moves. From whites perspective white chooses the best move of blacks best moves. We have to limit the amout of moves that minimax will go into the future due to the huge amount of possibilities that chess has. Therefore we stop the minimax algorithm at a certain depth.  

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

Initialize Global Cache initiallized in Main notebook `Play.ipynb`

## MaxValue-function
This function looks for the maximum value that white can achieve from a position. Meanwhile black tries to minimize the value of the position. Therefore white will take the maximal value of the minimum value of the next move that can happen.

Note: The depth is subtracted to give a better/worse rating to evaluate short mating paths as a petter path. (if there is a Mate in 3 (M3) and M1 Path then choose the M1 Path)

`maxValue(State::Board, depth::Int64)` is the **non-incremental** implementation of the maxValue function. It takes in the current `State` (Type `Board`) and the max-`depth` (Type `Int64`) it should calculate and returns the maximal score it can achieve.

In [ ]:
function maxValue(State::Board, depth::Int64)::Int64
    if isterminal(State) || depth == 0
        return evaluate_position(State) - depth
    end
    return maximum([ minValue(domove(State, ns), depth-1) for ns in moves(State) ])
end

`maxValue(State::Board, score, depth::Int64)` is the **incremental** implementation of the maxValue function. It takes in the current `State` (Type `Board`), the current centipawn-`score` of the board and the max-`depth` (Type `Int64`) it should calculate and returns the maximal score it can achieve.

In [ ]:
function maxValue(State::Board, score::Int64, depth::Int64)::Int64
    if isterminal(State) 
        return terminal_evaluation(State) - depth
    end
    if depth == 0
        return score
    end
    value = -Inf
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = max(minValue(State, nextEval, depth-1), value)
        undomove!(State, undoinfo)
    end
    return value
end

### with Memoization

This function is the same maxValue function above. Additionally it will save the calculated score in a global Cache. It will retrieve the value if it has already been calculated. It therefore must already be in the cache.

In [ ]:
function maxValue(State::Board, score::Int64, depth::Int64, cache)::Int64
    # Memoization
    entry = ("maxValue", deepcopy(State), depth)
    if entry in keys(cache)
        return cache[entry]
    end
    if isterminal(State)
        result = terminal_evaluation(State) - depth
    elseif depth == 0
        result = score
    else
        result = -Inf
        for move in moves(State)
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            result = max(minValue(State, nextEval, depth-1, cache), result)
            undomove!(State, undoinfo)
        end
    end
    # Save in Cache
    merge!(cache, Dict(entry => result))
    return result
end

## MinValue-function
MinValue does the opposite of the MaxValue function. Therefore it looks for the minimum value that black can achieve from a position. Meanwhile white tries to maximize the value of the position. Therefore black will take the minimal value of the maximum value of the next move that can happen.

`minValue(State::Board, depth::Int64)` is the **non-incremental** implementation of the minValue function. It takes in the current `State` (Type `Board`) and the max-`depth` (Type `Int64`) it should calculate and returns the minimal score it can achieve.

In [ ]:
function minValue(State::Board, depth::Int64)::Int64
    if isterminal(State) || depth == 0
      return evaluate_position(State) + depth
    end
    return minimum([ maxValue(domove(State, ns), depth-1) for ns in moves(State) ])
end

`minValue(State::Board, score, depth::Int64)` is the **incremental** implementation of the minValue function. It takes in the current `State` (Type `Board`), the current centipawn-`score` of the board and the max-`depth` (Type `Int64`) it should calculate and returns the minimal score it can achieve.

In [ ]:
function minValue(State::Board, score::Int64, depth::Int64)::Int64
    if isterminal(State) 
        return terminal_evaluation(State) + depth
    end
    if depth == 0
        return score
    end
    value = Inf
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = min(maxValue(State, nextEval, depth-1), value)
        undomove!(State, undoinfo)
    end
    return value
end

### with Memoization

In [ ]:
function minValue(State::Board, score::Int64, depth::Int64, cache)::Int64
    # Memoization
    entry = ("minValue", deepcopy(State), depth)
    if entry in keys(cache)
        return cache[entry]
    end
    if isterminal(State) 
        result = terminal_evaluation(State) - depth
    elseif depth == 0
        result = score
    else
        result = Inf
        for move in moves(State)
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            result = min(maxValue(State, nextEval, depth-1, cache), result)
            undomove!(State, undoinfo)
        end
    end
    # Save in Cache
    merge!(cache, Dict(entry => result))
    return result
end

## Add Memoization

In [ ]:
@nbinclude("Memoization.ipynb")

Using `invoke` to overwrite the functions defined above. Q: https://discourse.julialang.org/t/overwriting-functions/404

In [ ]:
# maxValue(State::Board, score::Int64, depth::Int64) = invoke(memoize(maxValue(State::Board, score::Int64, depth::Int64), Tuple{Board, Int64, Int64}, State, score, depth))

In [ ]:
# minValue(State::Board, score::Int64, depth::Int64) = invoke(memoize(minValue(State::Board, score::Int64, depth::Int64), Tuple{Board, Int64, Int64}, State, score, depth))

## Minimax-function

The `minimax` function takes in a `State` (Type `Board`) and the max-`depth` the algorithm should calculate and returns a pair containing the centipawn-value of the best move and the board of the best move.

### non-inkremental implementation

In [ ]:
function minimax(State::Board, depth::Int64)::Int64
    next_moves = moves(State)
    if sidetomove(State) == WHITE
        bestVal = maxValue(State, depth)
        BestMoves = [move for move in next_moves if minValue(domove(State, move), depth-1) == bestVal]
    elseif sidetomove(State) == BLACK
        bestVal = minValue(State, depth)
        BestMoves = [move for move in next_moves if maxValue(domove(State, move), depth-1) == bestVal]
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

### incremental implementation 

In [ ]:
function minimax(State::Board, score::Int64, depth::Int64, cache=Dict())::Tuple{Int64, Move}
    BestMoves = []
    bestVal = 0
    if sidetomove(State) == WHITE
        bestVal = maxValue(State, score, depth, cache)
        for move in moves(State)
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if minValue(State, nextEval, depth-1, cache) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    elseif sidetomove(State) == BLACK
        bestVal = minValue(State, score, depth, cache)
        for move in moves(State)
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if maxValue(State, nextEval, depth-1, cache) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end